In [1]:
import pandas as pd
from ANNbuilder import build_network
from pso import pso_optimization
from loss import mse_loss
from sklearn.metrics import classification_report

In [2]:
data = pd.read_csv("banknote_authentication.csv", header=None)
data.head()

,0,1,2,3,4
0,3.62160,8.6661,-2.8073,-0.44699,0
1,4.54590,8.1674,-2.4586,-1.46210,0
2,3.86600,-2.6383,1.9242,0.10645,0
3,3.45660,9.5228,-4.0112,-3.59440,0
4,0.32924,-4.4552,4.5718,-0.98880,0


In [3]:
data.shape

(1372, 5)

In [4]:
data.columns

Index([0, 1, 2, 3, 4], dtype='int64')

In [5]:
X = data.drop(4, axis=1).values
y = data[4].values.reshape(-1, 1)

In [6]:
X[:5]

array([[ 3.6216 ,  8.6661 , -2.8073 , -0.44699],
       [ 4.5459 ,  8.1674 , -2.4586 , -1.4621 ],
       [ 3.866  , -2.6383 ,  1.9242 ,  0.10645],
       [ 3.4566 ,  9.5228 , -4.0112 , -3.5944 ],
       [ 0.32924, -4.4552 ,  4.5718 , -0.9888 ]])

In [7]:
# Data separation and scaling
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [9]:
# Defining the network architecture and activation functions
architecture = [4, 8, 1]  # Network architecture
activations = ['relu', 'logistic']  # Activation functions

# Use the build_network function to create the network
network = build_network(architecture, activations)

# Get the dimensionality of the network weights vector
dimensions = network.total_weights()

# Call pso_optimization function with the created network and parameters
best_weights = pso_optimization(
    num_particles=50,
    num_iterations=300,
    loss_func=mse_loss,
    network=network,
    train_data=X_train_scaled,
    train_labels=y_train,
    dimensions=dimensions)

# Set the best weights to the network
network.set_weights(best_weights)

# Test the network
output = network.forward(X_test_scaled)
predictions = (output > 0.5).astype(int)

# Print the results
print(classification_report(y_test, predictions))

Iteration 25/300, Current Best Loss: 0.03006511062361988
Iteration 50/300, Current Best Loss: 0.004654219048031615
Iteration 75/300, Current Best Loss: 0.002589184775738911
Iteration 100/300, Current Best Loss: 7.150483924442828e-06
Iteration 125/300, Current Best Loss: 1.6912859061244844e-10
Iteration 150/300, Current Best Loss: 1.6912859061244844e-10
Iteration 175/300, Current Best Loss: 1.6912859061244844e-10
Iteration 200/300, Current Best Loss: 1.6518846255136701e-12
Iteration 225/300, Current Best Loss: 8.381927088513221e-13
Iteration 250/300, Current Best Loss: 2.7643623415401346e-13
Iteration 275/300, Current Best Loss: 2.2674668266009894e-13
Iteration 300/300, Current Best Loss: 9.35565370874332e-14
Final Best Loss: 9.35565370874332e-14 on Iteration: 298
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       148
           1       1.00      1.00      1.00       127

    accuracy                           1.00       275
   macro